# NLP for Production (industry/real-world)

In [2]:
import os
os.getcwd()

'/Users/cory/Desktop/Lectures/notebooks/nlp'

In [3]:
os.chdir("../../")
os.getcwd()

'/Users/cory/Desktop/Lectures'

# 1. Leveraging object orientied code



## 1.A Basic example

In [13]:
class SimpleOperationPerformer:
    def perform_string_print(self, input_string):
        print("Running perform_string_print...")
        result = input_string
        print(f"The result of string_print function: {result}")
        return result

    def perform_addition(self, num1, num2, num3):
        print("Running perform_addition...")
        final = num1 + num2 + num3
        print(f"The result of add_func function: {final}")
        return final

    def run_all_operations(self):
        print("\nStarting run_all_operations...")
        self.perform_string_print('LOOK MOM I CAN PRINT')
        self.perform_addition(5, 3, 2)
        print("Finished run_all_operations.")

# Let's create our SimpleOperationPerformer tool
ot = SimpleOperationPerformer()

In [15]:
# Now, let's tell it to run each action individually
print("\nRunning operations individually:")
ot.perform_string_print('Hello from individual call!')
ot.perform_addition(10, 20, 30)


Running operations individually:
Running perform_string_print...
The result of string_print function: Hello from individual call!
Running perform_addition...
The result of add_func function: 60


60

In [ ]:
# And let's also run all operations together
ot.run_all_operations()


Starting run_all_operations...
Running perform_string_print...
The result of string_print function: LOOK MOM I CAN PRINT
Running perform_addition...
The result of add_func function: 10
Finished run_all_operations.


## 1.1 Data pipeline

In [ ]:
import pandas as pd
import requests
import io

class DataProcessor:
    # '__init__' is a special function that runs automatically when you make a DataProcessor tool.
    def __init__(self, intermediate_folder="intermediate/", raw_folder="raw/"):
        # Inside '__init__', we're giving our new startup tool some initial settings.
        # We're setting up where this tool will look for and store its information.
        self.intermediate_folder = intermediate_folder  # This tells the tool where to put partially finished info.
        self.raw_folder = raw_folder                    # This tells the tool where the original info is kept.
        self.df_macro = None                             # This will hold macro info after we download it.
        self.df_oecd = None                              # This will hold OECD info after we download it.

    # This is a new ability for our tool: downloading macro info from a website.
    def download_macro(self, macro_url):
        self.df_macro = pd.read_stata(macro_url) # We use a special tool (pd.read_stata) to read the info directly from the web address.
        # Once downloaded, we store the info within our tool and then give it back.
        return self.df_macro

    # Here's another new ability: downloading info from the OECD website.
    def download_oecd(self, oecd_url):
        
        response = requests.get(oecd_url) # We ask the website for the info.
        response.raise_for_status() # If something went wrong while asking, this will let us know.
        data = io.StringIO(response.text) # We take the text info we got and treat it like a file.
        self.df_oecd = pd.read_csv(data) # Then, we use another special tool (pd.read_csv) to understand this info.

        return self.df_oecd # We store this info in our tool and then give it back.

# Let's say you want to create a DataProcessor startup tool to organize your files and download data.
# When you do this: The '__init__' function automatically runs, setting up the initial storage locations
dp = DataProcessor()

In [19]:
macro_df = dp.download_macro(
    macro_url = 'https://github.com/KMueller-Lab/Global-Macro-Database/raw/refs/heads/main/data/final/chainlinked_infl.dta'
)
dp.df_macro.head(2)

,ISO3,year,ADB_infl,AHSTAT_infl,AMF_infl,BCEAO_infl,BIS_infl,BORDO_infl,CEPAC_infl,EUS_infl,...,OECD_KEI_infl,WB_CC_infl,WDI_infl,WDI_ARC_infl,CS1_infl,CS2_infl,infl,chainlinking_ratio,source,source_change
0,ZWE,2029.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.131044,1.0,IMF_WEO,NaN
1,ZWE,2028.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.108963,1.0,IMF_WEO,NaN


In [20]:
oecd_df = dp.download_oecd(
    oecd_url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_PDB@DF_PDB_ULC_Q,1.0/.Q.......?startPeriod=1990-Q4&format=csv"
)
dp.df_oecd.head(2)

,DATAFLOW,REF_AREA,FREQ,MEASURE,ACTIVITY,UNIT_MEASURE,PRICE_BASE,TRANSFORMATION,ADJUSTMENT,CONVERSION_TYPE,TIME_PERIOD,OBS_VALUE,OBS_STATUS,UNIT_MULT,BASE_PER,DECIMALS
0,OECD.SDD.TPS:DSD_PDB@DF_PDB_ULC_Q(1.0),AUS,Q,ULCE,_T,PP,V,G1,S,NC,1990-Q4,-1.254723,A,0,NaN,2
1,OECD.SDD.TPS:DSD_PDB@DF_PDB_ULC_Q(1.0),AUS,Q,ULCE,_T,PP,V,G1,S,NC,1991-Q1,1.116150,A,0,NaN,2


## 1.2 Data pipeline advanced

 - from src.examples.data_processor python file

In [23]:
from src.examples.data_processor import DataProcessor

In [22]:
# Let's create our DataProcessor tool.
processor = DataProcessor()

# Now, let's define the web addresses for our data:
macro_data_url = 'https://github.com/KMueller-Lab/Global-Macro-Database/raw/refs/heads/main/data/final/chainlinked_infl.dta'
oecd_data_url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_PDB@DF_PDB_ULC_Q,1.0/.Q.......?startPeriod=1990-Q4&format=csv"

# Now, we will run each method of our 'processor' tool one by one.

# 1. Download macro data
macro_df = processor.download_macro(macro_data_url)

# 2. Download OECD data
oecd_df = processor.download_oecd(oecd_data_url)

# 3. Filter and rename macro data for New Zealand
macro_nz_df = processor.filter_rename_macro_nz()

# 4. Filter and rename OECD data for New Zealand
oecd_nz_df = processor.filter_rename_oecd_nz()

# 5. Convert datetime in macro data
macro_nz_df_dt = processor.convert_datetime_macro_nz()

# 6. Convert datetime in OECD data
oecd_nz_df_dt = processor.convert_datetime_oecd_nz()

# 7. Set index for macro data
macro_nz_df_indexed = processor.set_index_macro_nz()

# 8. Set index for OECD data
oecd_nz_df_indexed = processor.set_index_oecd_nz()

# 9. Merge the two datasets
merged_df = processor.merge_data()

# 10. Export the processed and raw data
processor.export_data()

Running download_macro...
Macro data downloaded successfully.
Running download_oecd...
OECD data downloaded successfully.
Running filter_rename_macro_nz...
Macro data filtered and renamed for NZ.
Running filter_rename_oecd_nz...
OECD data filtered and renamed for NZ.
Running convert_datetime_macro_nz...
Macro dates converted to datetime.
Running convert_datetime_oecd_nz...
OECD dates converted to datetime.
Running set_index_macro_nz...
Macro data index set to country and date.
Running set_index_oecd_nz...
OECD data index set to country and date.
Running merge_data...
Macro and OECD data merged.
Running export_data...
Merged data exported to: data/intermediate/processed/merged_data_nz.csv
Raw OECD data exported to: data/raw/oecd_raw.csv


# 2. Applying OOP code to LLMs

## 2.1 Running local LLMs (from your computer)

### 2.1.1 Ollama
- https://ollama.com/

In [27]:
from src.llm.inference.llm_inference_ollama import LLMInferenceOllama

In [28]:
llm = LLMInferenceOllama(model_name= "gemma3:270m")

In [29]:
models = llm.list_models()
for model in models['models']:
    print(f"{model['model']}")

llava:7b
mistral:7b
gemma3:270m
qwen3:0.6b
deepseek-r1:70b
gemma3:27b
llama3.1:8b
qwen3:32b
qwen2.5-coder:32b
magistral:latest
llama3.2:latest
deepseek-r1:32b
gpt-oss:20b


### 2.1.2 LMstudio 
- https://lmstudio.ai/

In [37]:
from src.llm.inference.llm_inference_lmstudio import LLMInferenceLMStudio
llm = LLMInferenceLMStudio(model_name="liquid/lfm2-1.2b")
llm.check_model_loaded()
prompt = "資本規制について教えてください。箇条書きで回答してください。"
system_message = "あなたは経済学者です。"
llm.load_prompt(prompt, system_message=system_message, verbose=True)
result = llm.infer(max_tokens_response=300)
print(result)

Checking model: liquid/lfm2-1.2b
Loaded models: ['liquid/lfm2-1.2b']
Model loaded in memory: True
Prompt Length: 30
資本規制について以下に経済学的な観点から回答します。

- **定義**: 資本規制とは、金融市場における資金の流れを管理・制限する政府の政策や法律を指します。これは、金融安定の維持、金融危機の防止、経済成長の促進を目的としています。

- **目的**:
  - **金融安定の維持**: 資本の急激な流入や流出による市場の不安定化を防ぎ、金融システムの健全性を保つ。
  - **経済成長の促進**: 適切な資本の流れを確保することで、投資機会の拡大や経済活動の活性化を図る。
  - **不正行為の防止**: マネーロンダリングやテロ資金供与などの違法行為を抑止する。
  - **為替レートの安定**: 過度な資本移動による為替レートの急激な変動を防ぐ。

- **種類**:
  - **流出規制**: 資本が国外に急速に流出するのを防ぐための規制（例：外貨持ち出し制限）。
  - **流入規制**: 外国からの資本流入を管理するための規制（例：投資上限の設定）。
  -


## 2.2 Running LLMs via Groq API (similar to Gemini, Claude, etc)

- Note groq is not the api affiliated with X's Grok LLM
    - https://groq.com/pricing

In [41]:
from src.llm.inference.llm_inference_groqinference import LLMInferenceGroqInference
llm = LLMInferenceGroqInference(model_name="llama-3.3-70b-versatile")
prompt = "資本規制について教えてください。箇条書きで回答してください。"
system_message = "あなたは経済学者です。"
llm.load_prompt(prompt, system_message=system_message, verbose=True)

Prompt Length: 30


In [42]:
result = llm.infer(max_tokens_response=300)
print(result)

資本規制については以下の点が重要です。

*   資本規制は、銀行や金融機関が保有する資本の水準を規定することで、金融の安定性とリスクを管理することを目的としています。
*   バーゼル協定が資本規制の国際基準として広く採用されており、銀行が最低限の資本規制を満たす必要があります。
*   資本規制により、銀行はリスクを抑制し、経済の変動に耐える能力を高めることができます。
*   また、資本規制は金融の安定性を維持し、金融危機の発生を防ぐ役割も果たしています。
*   ただし、資本規制には、金融機関がより厳格な規制に従わなければならないため、融資の柔軟性が制限される可能性があります。
*   さらに、規制の厳格性によって、金融機関がより慎重な貸し出しを行うようになり、経済の成長が制限される可能性もあります。
*   資本規制は、金融機関と経済の成長をバランスさせる必要があるため、調整が必要です。


## 2.3 Use previous two OOP coding frameworks in more agnostic pipeline runner

In [49]:
from src.llm.llm_inference_runner import LLMInferenceRunner

### 2.3.1 Run from lm studio

In [44]:
runner = LLMInferenceRunner(
    llm_provider='lmstudio',
    model_name='liquid/lfm2-1.2b',
    save_path='./test_results'
)

prompt = "Classify this text according to topic: {text}"
system_message = "You are a helpful assistant"

runner.set_prompt(prompt, system_message=system_message, verbose=True)

test_inputs = [
    "The quick brown fox jumps over the lazy dog. This is a common pangram used in typography.",
    "Machine learning is a subset of artificial intelligence that enables computers to learn from data.",
    "Climate change is affecting weather patterns around the world, causing more extreme weather events."
]

print("\nRunning batch inference...")
results = runner.infer(test_inputs, seconds_delay=1, save_intermediate_n=2)

print("\n=== Results ===")
for i, (inp, out) in enumerate(zip(test_inputs, results)):
    print(f"\nInput {i+1}: {inp[:50]}...")
    print(f"Output {i+1}: {out}")

Prompt Length: 45

Running batch inference...


 67%|██████▋   | 2/3 [00:02<00:01,  1.14s/it]

Progress saved: 2/3 items processed


100%|██████████| 3/3 [00:03<00:00,  1.14s/it]


=== Results ===

Input 1: The quick brown fox jumps over the lazy dog. This ...
Output 1: Topic: Typography and Pangrams

Input 2: Machine learning is a subset of artificial intelli...
Output 2: Topic: Artificial Intelligence and Machine Learning.

Input 3: Climate change is affecting weather patterns aroun...
Output 3: Topic: Climate Change and Extreme Weather Events.


### 2.3.2 Run from groq

In [47]:
print(runner.llm_provider)
print(runner.model_name)

lmstudio
liquid/lfm2-1.2b


In [50]:
runner.llm_provider = 'groqinference'
runner.model_name = 'llama-3.3-70b-versatile'

In [51]:
print(runner.llm_provider)
print(runner.model_name)

groqinference
llama-3.3-70b-versatile


In [54]:
runner.set_prompt(prompt, system_message=system_message, verbose=True)

Prompt Length: 45


In [55]:
results = runner.infer(test_inputs, seconds_delay=1, save_intermediate_n=2)
print("\n=== Results ===")
for i, (inp, out) in enumerate(zip(test_inputs, results)):
    print(f"\nInput {i+1}: {inp[:50]}...")
    print(f"Output {i+1}: {out}")

 67%|██████▋   | 2/3 [00:02<00:01,  1.17s/it]

Progress saved: 2/3 items processed


100%|██████████| 3/3 [00:03<00:00,  1.19s/it]


=== Results ===

Input 1: The quick brown fox jumps over the lazy dog. This ...
Output 1: This text is classified under the topic of "Typography" and "Language." More specifically, it's a pangram example.

Input 2: Machine learning is a subset of artificial intelli...
Output 2: Topic: Artificial Intelligence and Machine Learning

Input 3: Climate change is affecting weather patterns aroun...
Output 3: The text is classified under the topic of "Climate Change and Extreme Weather Events."


## 

# 3. MLOps/LLMOps Tech Stack

# 3.A Configuration Management Best Practices

- **OOP coding practices for configs**: Use inheritance patterns (base configs with environment/model-specific overrides) to manage shared parameters across pipeline stages (3.1), model variants (3.2), experiment sets (3.3), and LLM prompt templates (3.4)
- **YAML for pipeline and input tracking**: Define reproducible pipelines in DVC (3.1) with input data versions, log experiment parameters in W&B (3.3), configure model metadata in MLflow (3.2), and version LLM prompts in Langfuse (3.4) using YAML
- **Git version control for configurations**: Track all YAML configs, pipeline definitions, and code changes in Git to maintain audit trails, enable rollbacks, and coordinate changes across pipeline components (3.1), model versions (3.2), experiments (3.3), and LLM workflows (3.4)
- **Secrets and credentials separation**: Externalize API keys, database credentials, and cloud storage tokens from config files using environment variables or secret managers to safely version control configurations across all tools (3.1-3.4)



## 3.1 Pipeline Management (DVC)
- **Pipeline orchestration**: Define end-to-end ML pipelines with dependencies between data preprocessing, training, and evaluation stages
- **Reproducibility**: Lock data versions, code commits, and dependencies together to recreate exact training runs
- **Remote storage integration**: Track large datasets and models in S3/GCS/Azure without bloating Git repos

## 3.2 Model Registry & Versioning (MLflow/DVC)
- **Centralized model storage**: Register, version, and track model artifacts with metadata in a single source of truth
- **Stage transitions**: Promote models through staging/production with approval workflows and rollback capabilities
- **A/B testing support**: Deploy multiple model versions simultaneously for comparison and gradual rollout

## 3.3 Experiment Management (Weights & Biases)
- **Hyperparameter tracking**: Log all training configs, metrics, and results automatically for easy comparison across experiments
- **Real-time monitoring**: Visualize training metrics (loss, accuracy) live during runs to catch issues early
- **Artifact management**: Store model checkpoints, datasets, and predictions with full lineage tracking

## 3.4 LLMOps (Langfuse/LangChain)
- **Production observability**: Track LLM calls, token usage, latency, and costs with prompt versioning and performance analytics
- **Complex workflows**: Build multi-step LLM applications with chains, agents, and memory for reasoning tasks
- **Tool integration**: Connect LLMs to external APIs, databases, and vector stores for retrieval-augmented generation